# Plans for the model

## Architecture:

#### Architecture will be simillar to transformer, as we will first predict country and then pass it into predicting the city

#### Let's call the model predicting the countries a 'country block' and cities a 'city block'

#### Each block will be made out of a mix of RNN and CNN layers, and ending with a few Linear layers

## Training:

#### First we will teach the model to predict the destination country correctly

#### Next we will get true coutnry data and teach the model to predict cities

#### Finnaly we will combine it and teach the 2 blocks to work together

## Shapes:

#### Input shape of country block: (batch_size, 9)

#### Input shape of city block: (batch_size, 10), with the last element being the output of country block

#### Output shape of country block: (batch_size, 192)

#### Output shape of city block: (batch_size, 37615)

## Technologies:

#### 1. PyTorch

### Note that this is the first version of the model and it will change as I perform more tests

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class CountryBlock(nn.Module):
    def __init__(self):
        super(CountryBlock, self).__init__()
        self.lstm_1 = nn.LSTM(1, 32, 1, batch_first=True)
        self.lstm_2 = nn.LSTM(32, 64, 1, batch_first=True)
        self.lstm_3 = nn.LSTM(64, 96, 1, batch_first=True)

        self.conv_1 = nn.Conv1d(96, 128, 1, 1)
        self.conv_2 = nn.Conv1d(128, 64, 1, 1)
        self.conv_3 = nn.Conv1d(64, 32, 1, 1)

        self.flatten = nn.Flatten()

        self.linear_1 = nn.Linear(32 * 9, 256)
        self.linear_2 = nn.Linear(256, 192)

        self.relu = nn.ReLU()
        self.drop_02 = nn.Dropout(0.2)
        self.drop_05 = nn.Dropout(0.5)

    def forward(self, X):
        X, _ = self.lstm_1(X)
        X = self.drop_02(X)

        X, _ = self.lstm_2(X)
        X = self.drop_02(X)

        X, _ = self.lstm_3(X)
        X = self.drop_02(X)

        X = X.permute(0, 2, 1)

        X = self.conv_1(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.conv_2(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.conv_3(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.flatten(X)

        X = self.linear_1(X)
        X = self.relu(X)
        X = self.drop_05(X)
        X = self.linear_2(X)

        X = F.softmax(X)

        return X

In [4]:
# Testing the block
block = CountryBlock()
block.forward(torch.rand(1, 9, 1))
# print(block)

C:\Users\Marcel\AppData\Local\Temp\ipykernel_26356\2971947118.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(X)


tensor([[0.0054, 0.0055, 0.0053, 0.0055, 0.0050, 0.0053, 0.0050, 0.0049, 0.0054,
         0.0049, 0.0050, 0.0055, 0.0051, 0.0053, 0.0053, 0.0051, 0.0049, 0.0056,
         0.0053, 0.0052, 0.0049, 0.0053, 0.0052, 0.0051, 0.0053, 0.0055, 0.0053,
         0.0052, 0.0052, 0.0053, 0.0050, 0.0054, 0.0056, 0.0050, 0.0051, 0.0050,
         0.0047, 0.0051, 0.0050, 0.0052, 0.0050, 0.0053, 0.0053, 0.0050, 0.0055,
         0.0055, 0.0051, 0.0052, 0.0052, 0.0048, 0.0052, 0.0051, 0.0052, 0.0052,
         0.0050, 0.0050, 0.0057, 0.0051, 0.0054, 0.0053, 0.0057, 0.0052, 0.0051,
         0.0056, 0.0053, 0.0050, 0.0051, 0.0055, 0.0054, 0.0054, 0.0053, 0.0053,
         0.0056, 0.0051, 0.0058, 0.0050, 0.0048, 0.0053, 0.0052, 0.0053, 0.0051,
         0.0053, 0.0056, 0.0050, 0.0054, 0.0049, 0.0056, 0.0053, 0.0056, 0.0052,
         0.0057, 0.0052, 0.0050, 0.0051, 0.0054, 0.0051, 0.0053, 0.0051, 0.0051,
         0.0050, 0.0054, 0.0049, 0.0055, 0.0055, 0.0055, 0.0051, 0.0048, 0.0051,
         0.0053, 0.0053, 0.0

In [5]:
class CityBlock(nn.Module):
    def __init__(self):
        super(CityBlock, self).__init__()
        self.lstm_1 = nn.LSTM(1, 32, 1, batch_first=True)
        self.lstm_2 = nn.LSTM(32, 64, 1, batch_first=True)
        self.lstm_3 = nn.LSTM(64, 96, 1, batch_first=True)

        self.conv_1 = nn.Conv1d(96, 128, 1, 1)
        self.conv_2 = nn.Conv1d(128, 256, 1, 1)
        self.conv_3 = nn.Conv1d(256, 512, 1, 1)

        self.flatten = nn.Flatten()

        self.linear_1 = nn.Linear(512 * 10, 512 * 20)
        self.linear_2 = nn.Linear(512 * 20, 37615)

        self.relu = nn.ReLU()
        self.drop_02 = nn.Dropout(0.2)
        self.drop_05 = nn.Dropout(0.5)

    def forward(self, X):
        X, _ = self.lstm_1(X)
        X = self.drop_02(X)

        X, _ = self.lstm_2(X)
        X = self.drop_02(X)

        X, _ = self.lstm_3(X)
        X = self.drop_02(X)

        X = X.permute(0, 2, 1)

        X = self.conv_1(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.conv_2(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.conv_3(X)
        X = self.relu(X)
        X = self.drop_02(X)

        X = self.flatten(X)

        X = self.linear_1(X)
        X = self.relu(X)
        X = self.drop_05(X)

        X = self.linear_2(X)

        X = F.softmax(X)

        return X

In [6]:
# Testing the block
block = CityBlock()
block.forward(torch.rand(1, 10, 1))
print(block)

CityBlock(
  (lstm_1): LSTM(1, 32, batch_first=True)
  (lstm_2): LSTM(32, 64, batch_first=True)
  (lstm_3): LSTM(64, 96, batch_first=True)
  (conv_1): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
  (conv_2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  (conv_3): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_1): Linear(in_features=5120, out_features=10240, bias=True)
  (linear_2): Linear(in_features=10240, out_features=37615, bias=True)
  (relu): ReLU()
  (drop_02): Dropout(p=0.2, inplace=True)
  (drop_05): Dropout(p=0.5, inplace=True)
)


C:\Users\Marcel\AppData\Local\Temp\ipykernel_26356\718956853.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(X)


In [25]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.country_block = CountryBlock()
        self.city_block = CityBlock()

        self.country_embedding = nn.Embedding(195, 1)
        self.city_embedding = nn.Embedding(39900, 1)

    def forward(self, X):
        X[:, 0] = self.city_embedding(X[:, 0].long()).squeeze(2)
        X[:, 1] = self.country_embedding(X[:, 1].long()).squeeze(2)
        X[:, 2] = self.country_embedding(X[:, 2].long()).squeeze(2)

        country = self.country_block(X)

        country_argmax = torch.argmax(country, dim=1, keepdim=True)
        
        country_embeddeed = self.country_embedding(country_argmax)

        X = torch.cat((X, country_embeddeed), dim=1)

        city = self.city_block(X)

        return country, city

In [26]:
# Testing the model
city = torch.randint(0, 39900, (1024, 1, 1))
country_1 = torch.randint(0, 195, (1024, 1, 1))
country_2 = torch.randint(0, 195, (1024, 1, 1))

print(torch.max(country_1))
print(torch.max(country_2))

X = torch.cat((city, country_1, country_2), dim=1)

random_values = torch.rand(1024, 6, 1)

X = torch.cat((X, random_values), dim=1)

model = Model()
model.forward(X)
print(model)

tensor(194)
tensor(194)


C:\Users\Marcel\AppData\Local\Temp\ipykernel_26356\2971947118.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(X)


Model(
  (country_block): CountryBlock(
    (lstm_1): LSTM(1, 32, batch_first=True)
    (lstm_2): LSTM(32, 64, batch_first=True)
    (lstm_3): LSTM(64, 96, batch_first=True)
    (conv_1): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
    (conv_2): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    (conv_3): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear_1): Linear(in_features=288, out_features=256, bias=True)
    (linear_2): Linear(in_features=256, out_features=192, bias=True)
    (relu): ReLU()
    (drop_02): Dropout(p=0.2, inplace=True)
    (drop_05): Dropout(p=0.5, inplace=True)
  )
  (city_block): CityBlock(
    (lstm_1): LSTM(1, 32, batch_first=True)
    (lstm_2): LSTM(32, 64, batch_first=True)
    (lstm_3): LSTM(64, 96, batch_first=True)
    (conv_1): Conv1d(96, 128, kernel_size=(1,), stride=(1,))
    (conv_2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    (conv_3): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
 

C:\Users\Marcel\AppData\Local\Temp\ipykernel_26356\718956853.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(X)
